In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hlh
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge"
lustre_dir = "file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge"
import_lustre_dir="file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge"
sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()


pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/hail-20211112-1032-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [4]:
gwas=hl.read_table(f"{lustre_dir}/gwas_merged.table")
from bokeh.plotting import output_file, save, show


In [9]:
p = hl.plot.manhattan(gwas.p_value, title=f" GWAS")
output_file(
            f"{plot_dir}/gwas/gwas_merged_manhattan.html", mode='inline')
save(p)
print(f"Plotting QQ plot for ")
q = hl.plot.qq(gwas.p_value, collect_all=False,
                       n_divisions=100, title=f" QQ plot")
output_file(
            f"{plot_dir}/gwas/gwas_merged-QQplot.html", mode='inline')
save(q)

Plotting QQ plot for 


2021-11-12 10:38:08 Hail: INFO: Ordering unsorted dataset with network shuffle


'/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/gwas/gwas_merged-QQplot.html'

In [8]:
image = hl.plot.manhattan(gwas.p_value, title=f" GWAS")
show(image)


RuntimeError: Models must be owned by only a single document, LegendItem(id='1418', ...) is already in a doc

In [14]:
mt.count()

(48, 222108)

In [7]:
mt_annotated=mt.annotate_cols(cohort=(hl.case()
                .when(mt.s.startswith("EGAN"),"IBD")
                .default("UKBB")
                ))

In [21]:
mt_annotated=mt_annotated.annotate_cols(
pheno=
    (hl.case()
     .when(mt_annotated.cohort=="IBD",1)
     .when(mt_annotated.cohort=="UKBB",0)
     .default(0)
    )

)

In [23]:
mt_ibd=mt_annotated.filter_cols(mt_annotated.cohort=="IBD")
mt_ibd.count()

(48, 21465)

In [48]:
mt=hl.variant_qc(mt_annotated)

In [25]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))


Samples: 222108  Variants: 25


In [51]:
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))



Samples: 222108  Variants: 48


In [49]:
gwas = hl.linear_regression_rows(
        y=mt.pheno,
        x=mt.GT.n_alt_alleles(), covariates=[1.0], pass_through=[mt.rsid])


2021-11-08 16:17:15 Hail: INFO: linear_regression_rows: running on 222108 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...


In [52]:
gwas.row.describe()

--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh38>, 
        alleles: array<str>, 
        rsid: str, 
        n: int32, 
        sum_x: float64, 
        y_transpose_x: float64, 
        beta: float64, 
        standard_error: float64, 
        t_stat: float64, 
        p_value: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


In [53]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

In [30]:
p = hl.plot.qq(gwas.p_value)
show(p)

2021-11-08 15:28:01 Hail: INFO: Ordering unsorted dataset with network shuffle


In [55]:
gwas.export(
            f"{lustre_dir}/test_gwas_48.tsv", header=True)


2021-11-08 16:21:39 Hail: INFO: merging 20 files totalling 5.9K...
2021-11-08 16:21:39 Hail: INFO: while writing:
    file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/test_gwas_48.tsv
  merge time: 296.126ms


In [67]:
gwas_pandas=gwas.to_pandas()

In [60]:
hits=pd.read_csv("/home/ubuntu/data/scripts/cd2021_legit_hits.csv", sep=",")

In [79]:
import csv
def csv_to_bed(csv_file, bed_file):
    with open(csv_file, "r") as csvfile:
        #9," 136,464,447 ",C,T,0.0259,
        reader = csv.reader(csvfile, delimiter=",")

        headers = next(reader)[:1]

        for row in reader:
            if row[0].startswith("#"):
                continue
            if row[0].startswith(" "):
                continue
          

            chromosome = row[0]
            start = row[1]
            start=start.replace(',','')
            if start !='':
                stop=int(start)
                start=int(start)-1
                ref = row[2]
                alt = row[3]
                meta=row[45]
                if meta is not None:
                    print("Chromosome\tStart\tStop\tor_meta")
                    print(f"chr{chromosome}\t{start}\t{stop}\t{meta}\n")
                    with open(bed_file, 'a') as outfile:
                        outfile.write(f"chr{chromosome}\t{start}\t{stop}\t{meta}\n")



In [80]:
csv_to_bed("/home/ubuntu/data/scripts/cd2021_legit_hits.csv", "/home/ubuntu/data/scripts/cd2021_legit_hits_meta.tsv")


Chromosome	Start	Stop	or_meta
chr1	28150680	28150681	1.70195563

Chromosome	Start	Stop	or_meta
chr1	67206979	67206980	1.34157649

Chromosome	Start	Stop	or_meta
chr1	67240216	67240217	0.701929351

Chromosome	Start	Stop	or_meta
chr1	67240274	67240275	0.490653882

Chromosome	Start	Stop	or_meta
chr2	233263165	233263166	1.152225207

Chromosome	Start	Stop	or_meta
chr2	233320673	233320674	0.842061881

Chromosome	Start	Stop	or_meta
chr2	233527422	233527423	0.864139955

Chromosome	Start	Stop	or_meta
chr4	3447924	3447925	1.170496452

Chromosome	Start	Stop	or_meta
chr4	94573344	94573345	1.610480351

Chromosome	Start	Stop	or_meta
chr4	102267551	102267552	1.210603217

Chromosome	Start	Stop	or_meta
chr5	40681151	40681152	0.860101685

Chromosome	Start	Stop	or_meta
chr5	150848435	150848436	1.226284975

Chromosome	Start	Stop	or_meta
chr6	159041391	159041392	0.786150972

Chromosome	Start	Stop	or_meta
chr8	21909728	21909729	1.247943912

Chromosome	Start	Stop	or_meta
chr9	136365139	136365140	0.472240827



In [37]:
hits.head()

,Chr,Pos,A0,A1,gnomad_NFE_AF,FigureStatus,PIP,LDPop,ConditionalP,ConditionalVar,...,se_meta,p_nextera,p_twist,p_regeneron,p_sangerwgs,p_sangerwes,p_meta,StatusCode,Status,Status_comment
0,1.0,"28,150,681",T,C,0.0037,New Locus,NaN,NaN,NaN,NaN,...,0.076192,1.830000e-06,2.283318e-02,1.473547e-03,3.668090e-04,0.219340,2.960000e-12,1.0,New Locus,NaN
1,1.0,"67,206,980",T,C,0.0071,Tagging,NaN,NFE,0.000785,1:67129425_T_A,...,0.053681,4.290000e-05,8.672825e-02,2.375478e-01,8.523736e-03,0.095616,4.400000e-08,4.0,Tagging,NaN
2,1.0,"67,240,217",G,A,0.0167,Known causal candidate,0.9730,NaN,NaN,NaN,...,0.045934,5.800000e-07,2.760000e-05,1.666882e-01,1.277610e-04,0.046550,1.310000e-14,2.0,BestPIP,NaN
3,1.0,"67,240,275",G,A,0.0586,Known causal candidate,0.1018,NaN,NaN,NaN,...,0.028060,9.260000e-50,5.150000e-36,9.750000e-20,6.140000e-48,NaN,4.750000e-142,3.0,InCredibleSet,NaN
4,2.0,"233,263,166",A,G,0.0618,Tagging,NaN,ASJ,0.836767,2:233252802_G_A,...,0.021820,4.170580e-04,6.033530e-04,1.232527e-01,6.522439e-03,0.002091,8.360000e-11,4.0,Tagging,NaN


In [44]:
hits.or_meta

0      1.701956
1      1.341576
2      0.701929
3      0.490654
4      1.152225
         ...   
994         NaN
995         NaN
996         NaN
997         NaN
998         NaN
Name: or_meta, Length: 999, dtype: float64

In [41]:
hits.columns

Index(['Chr', ' Pos ', 'A0', 'A1', 'gnomad_NFE_AF', 'FigureStatus', 'PIP',
       'LDPop', 'ConditionalP', 'ConditionalVar', 'ConditionalLD(r)',
       'consequence', 'gene', 'gene_symbol', 'AminoAcidSub', 'AJ_MAF',
       'FIN_MAF', 'LIT_MAF', 'NFE_MAF', 'Effect', 'StdErr', 'P-value',
       'Direction', 'HetISq', 'HetChiSq', 'HetPVal', 'beta(TWIST.SAIGE)',
       'se(TWIST.SAIGE)', 'p(TWIST.SAIGE)', 'Case_AF0(TWIST.SAIGE)',
       'Case_AF1(TWIST.SAIGE)', 'Control_AF0(TWIST.SAIGE)',
       'Control_AF1(TWIST.SAIGE)', 'beta(Nextera.SAIGE)', 'se(Nextera.SAIGE)',
       'p(Nextera.SAIGE)', 'Case_AF0(Nextera.SAIGE)',
       'Case_AF1(Nextera.SAIGE)', 'Control_AF0(Nextera.SAIGE)',
       'Control_AF1(Nextera.SAIGE)', 'or_nextera', 'or_twist', 'or_regeneron',
       'or_sangerwgs', 'or_sangerwes', 'or_meta', 'se_nextera', 'se_twist',
       'se_regeneron', 'se_sangerwgs', 'se_sangerwes', 'se_meta', 'p_nextera',
       'p_twist', 'p_regeneron', 'p_sangerwgs', 'p_sangerwes', 'p_meta',
      

In [85]:
hits_pd=pd.read_csv("/home/ubuntu/data/scripts/cd2021_legit_hits_meta.tsv",sep="\t", header=None)

In [86]:
hits_pd.head()

,0,1,2,3
0,chr1,28150680,28150681,1.701956
1,chr1,67206979,67206980,1.341576
2,chr1,67240216,67240217,0.701929
3,chr1,67240274,67240275,0.490654
4,chr2,233263165,233263166,1.152225


In [68]:
gwas_pandas.head()

,locus.contig,locus.position,alleles,rsid,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
0,chr1,28150681,"[T, C]",chr1_28150681_T_C,222108,1180.000000,152.000000,0.032124,0.008595,3.737584,1.858439e-04
1,chr1,67206980,"[T, C]",chr1_67206980_T_C,222108,3360.060512,367.030256,0.012700,0.005119,2.480856,1.310745e-02
2,chr1,67240217,"[G, A]",chr1_67240217_G_A;chr1_67240217_G_T,222108,5764.439612,405.583152,-0.026559,0.003912,-6.789678,1.126649e-11
3,chr1,67240275,"[G, A]",chr1_67240275_G_A,222108,27107.113104,1464.424285,-0.045186,0.001845,-24.485649,3.150032e-132
4,chr2,233263166,"[A, G]",chr2_233263166_A_G,222108,23241.778910,2358.209284,0.005067,0.001987,2.550347,1.076224e-02


In [91]:
merged_pd=pd.merge(gwas_pandas, hits_pd, left_on=['locus.contig','locus.position'], right_on=[0,2], how="left")

In [92]:
merged_pd.head()

,locus.contig,locus.position,alleles,rsid,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value,0,1,2,3
0,chr1,28150681,"[T, C]",chr1_28150681_T_C,222108,1180.000000,152.000000,0.032124,0.008595,3.737584,1.858439e-04,chr1,28150680,28150681,1.701956
1,chr1,67206980,"[T, C]",chr1_67206980_T_C,222108,3360.060512,367.030256,0.012700,0.005119,2.480856,1.310745e-02,chr1,67206979,67206980,1.341576
2,chr1,67240217,"[G, A]",chr1_67240217_G_A;chr1_67240217_G_T,222108,5764.439612,405.583152,-0.026559,0.003912,-6.789678,1.126649e-11,chr1,67240216,67240217,0.701929
3,chr1,67240275,"[G, A]",chr1_67240275_G_A,222108,27107.113104,1464.424285,-0.045186,0.001845,-24.485649,3.150032e-132,chr1,67240274,67240275,0.490654
4,chr2,233263166,"[A, G]",chr2_233263166_A_G,222108,23241.778910,2358.209284,0.005067,0.001987,2.550347,1.076224e-02,chr2,233263165,233263166,1.152225


In [93]:
merged_pd=merged_pd.drop([0,1,2], axis=1)

In [94]:
merged_pd.head()

,locus.contig,locus.position,alleles,rsid,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value,3
0,chr1,28150681,"[T, C]",chr1_28150681_T_C,222108,1180.000000,152.000000,0.032124,0.008595,3.737584,1.858439e-04,1.701956
1,chr1,67206980,"[T, C]",chr1_67206980_T_C,222108,3360.060512,367.030256,0.012700,0.005119,2.480856,1.310745e-02,1.341576
2,chr1,67240217,"[G, A]",chr1_67240217_G_A;chr1_67240217_G_T,222108,5764.439612,405.583152,-0.026559,0.003912,-6.789678,1.126649e-11,0.701929
3,chr1,67240275,"[G, A]",chr1_67240275_G_A,222108,27107.113104,1464.424285,-0.045186,0.001845,-24.485649,3.150032e-132,0.490654
4,chr2,233263166,"[A, G]",chr2_233263166_A_G,222108,23241.778910,2358.209284,0.005067,0.001987,2.550347,1.076224e-02,1.152225


In [97]:
merged_pd=merged_pd.rename(columns={3:"or_meta"})

In [98]:
merged_pd.head()

,locus.contig,locus.position,alleles,rsid,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value,or_meta
0,chr1,28150681,"[T, C]",chr1_28150681_T_C,222108,1180.000000,152.000000,0.032124,0.008595,3.737584,1.858439e-04,1.701956
1,chr1,67206980,"[T, C]",chr1_67206980_T_C,222108,3360.060512,367.030256,0.012700,0.005119,2.480856,1.310745e-02,1.341576
2,chr1,67240217,"[G, A]",chr1_67240217_G_A;chr1_67240217_G_T,222108,5764.439612,405.583152,-0.026559,0.003912,-6.789678,1.126649e-11,0.701929
3,chr1,67240275,"[G, A]",chr1_67240275_G_A,222108,27107.113104,1464.424285,-0.045186,0.001845,-24.485649,3.150032e-132,0.490654
4,chr2,233263166,"[A, G]",chr2_233263166_A_G,222108,23241.778910,2358.209284,0.005067,0.001987,2.550347,1.076224e-02,1.152225


In [100]:
merged_pd.to_csv("/home/ubuntu/data/scripts/gwas_test_or_meta.tsv",sep="\t",index=False)